In [84]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date

In [85]:
# Importing the cleaned/combined data from the csv

firefile = './output/all_raw_fire_data.csv'
df = pd.read_csv(firefile, skip_blank_lines=True, encoding='UTF-8', parse_dates=True,infer_datetime_format=True)
len(df)

/Users/lauraweislo/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


177738

In [86]:
# Looking at the nulls for the full dataset
df.isna().sum()

OBJECTID                         0
address                          0
address2                     69957
apt_room                     76722
arrive_date_time             33736
cleared_date_time             1526
dispatch_date_time            1454
exposure                         0
incident_number                  0
incident_type                 3101
incident_type_description     3101
platoon                       1560
station                       2358
Longitude                        0
Latitude                         0
dtype: int64

In [87]:
df['dispatch_date_time'] = df['dispatch_date_time']/1000
df['arrive_date_time'] = df['arrive_date_time']/1000
df['cleared_date_time'] = df['cleared_date_time']/1000

In [88]:
df['dispatch_date_time'] = pd.to_datetime(df['dispatch_date_time'], errors='coerce', yearfirst=True, unit='s')
df['arrive_date_time'] = pd.to_datetime(df['arrive_date_time'], errors='coerce', yearfirst=True, unit='s')
df['cleared_date_time'] = pd.to_datetime(df['cleared_date_time'], errors='coerce', yearfirst=True, unit='s')

In [89]:
# Making a year column for merging Population data
# df['Year'] = pd.DatetimeIndex(df['dispatch_date_time']).year
df.head()

,OBJECTID,address,address2,apt_room,arrive_date_time,cleared_date_time,dispatch_date_time,exposure,incident_number,incident_type,incident_type_description,platoon,station,Longitude,Latitude
0,474765,"6647 LAKE HILL DR RALEIGH, NC 27601",NaN,0,2007-11-15 11:17:00,2007-11-15 11:25:00,2007-11-15 11:10:00,0,07-0031665,NaN,NaN,,NaN,-78.6266,35.8702
1,474766,"539 METHOD RD RALEIGH, NC 27606",NaN,,2008-06-29 06:20:00,2008-06-29 06:42:00,2008-06-29 06:17:00,0,08-0017918,NaN,NaN,,NaN,-78.6936,35.7929
2,474767,"2100 RUARK CT RALEIGH, NC 27601",NaN,202,2008-11-18 04:19:00,2008-11-18 04:24:00,2008-11-18 04:12:00,0,08-0032426,NaN,NaN,,NaN,-78.6278,35.8122
3,474768,"1216 BEVERLY DR RALEIGH, NC 27601",NaN,,2007-08-21 22:52:00,2007-08-21 22:58:00,2007-08-21 22:47:00,0,07-0023051,444.0,Power line down,A,12.0,-78.5954,35.7612
4,474769,"8332 SHILOH CREEK CT RALEIGH, NC 27603",NaN,,2013-10-19 21:52:00,2013-10-19 21:59:00,2013-10-19 21:47:00,0,13-0030918,150.0,Outside rubbish fire Other,B,22.0,-78.5373,35.8916


In [90]:
df['Year'] = pd.DatetimeIndex(df['dispatch_date_time']).year

In [91]:
years = df.groupby(['Year']).OBJECTID.count()
years

Year
2007.0    13944
2008.0    14079
2009.0    13166
2010.0    13699
2011.0    14035
2012.0    13635
2013.0    13965
2014.0    14566
2015.0    15474
2016.0    16130
2017.0    18439
2018.0    15137
2019.0       15
Name: OBJECTID, dtype: int64

In [92]:
# Making a date column for merging the weather data later
df['Date'] = df['dispatch_date_time'].astype(str).str.split(' ',expand=True)[0]

In [93]:
df = df.loc[df['Year'] > 2007]
len(df)

162340

In [94]:
df = df.loc[df['Year'] < 2019]
len(df)

162325

In [95]:
df = df.loc[df['incident_type'].notnull()]

In [96]:
len(df)

159803

In [97]:
df.head()

,OBJECTID,address,address2,apt_room,arrive_date_time,cleared_date_time,dispatch_date_time,exposure,incident_number,incident_type,incident_type_description,platoon,station,Longitude,Latitude,Year,Date
4,474769,"8332 SHILOH CREEK CT RALEIGH, NC 27603",NaN,,2013-10-19 21:52:00,2013-10-19 21:59:00,2013-10-19 21:47:00,0,13-0030918,150.0,Outside rubbish fire Other,B,22.0,-78.5373,35.8916,2013.0,2013-10-19
5,474770,"7513 IDOLBROOK LN RALEIGH, NC 27615",NaN,,2014-02-14 21:49:00,2014-02-14 21:51:00,2014-02-14 21:46:00,0,14-0004846,510.0,Person in distress Other,A,4.0,-78.6476,35.8782,2014.0,2014-02-14
6,474771,"1205 S BLOODWORTH ST RALEIGH, NC 27601",NaN,,2012-04-29 05:17:00,2012-04-29 05:38:00,2012-04-29 05:16:00,0,12-0011779,440.0,Electrical wiring/equipment problem Other,A,1.0,-78.634,35.7654,2012.0,2012-04-29
7,474772,"7628 VALVIEW ST RALEIGH, NC 27601",NaN,,2012-04-28 23:46:00,2012-04-28 23:52:00,2012-04-28 23:43:00,0,12-9010131,561.0,Unauthorized burning,A,18.0,-78.6914,35.8844,2012.0,2012-04-28
8,474773,"4201 MANGROVE DR RALEIGH, NC 27603",NaN,,2014-01-26 22:35:00,2014-01-26 22:38:00,2014-01-26 22:31:00,0,14-0002916,520.0,Water problem Other,B,28.0,-78.5166,35.8727,2014.0,2014-01-26


In [98]:
df.to_csv("output/all_with_no_incident_nulls.csv", index=False, header=True)